In [3]:
import pandas as pd

# Projects Selected

In [104]:
# projects with git
project_git = pd.read_csv('../test-projects/historical_projects.csv')
project_git['project'] = project_git['project'].map(lambda x:x.lower())

# specify sha for each project
historical_rerun_flaky_tests = pd.read_csv('../test-projects/historical_rerun_flaky_tests.csv')
print('Flaky test shape:', historical_rerun_flaky_tests.shape)
# sha of project with the most test methods
projects_sha = historical_rerun_flaky_tests.groupby(by='project').apply(lambda x:x['sha'].value_counts().index[0])

# join git with sha
projects = pd.concat([projects_sha, project_git.set_index('project')], axis=1, join='inner').reset_index().rename(columns={0:'sha'})
print('Selected Project shape:',projects.shape)
projects.head()
projects.to_csv('../test-projects/projects.csv', index=None)

Flaky test shape: (5075, 4)
Selected Project shape: (22, 3)


In [108]:
# Remove flaky test detected in other sha
flaky_tests = historical_rerun_flaky_tests[historical_rerun_flaky_tests['sha'].isin(projects_sha)]
print('Flaky tests shape:', flaky_tests.shape)
flaky_tests.head()
# flaky_tests.to_csv('../test-projects/flaky_tests.csv', index=None)

Flaky tests shape: (903, 4)


# Flakify Results

In [13]:
flakify_results_IDoFT = pd.read_csv('../Flakify/results/Flakify_per_project_results_on_IDoFT_dataset.csv', header=0)
flakify_results_IDoFT['project_name'] = flakify_results_IDoFT['project_name'].map(lambda x:x.lower())
flakify_results_flakeflagger = pd.read_csv('../Flakify/results/Flakify_per_project_results_on_FlakeFlagger_dataset.csv', header=0)
flakify_results_flakeflagger['project_name'] = flakify_results_flakeflagger['Project Name'].map(lambda x:x.lower())
flakify_results_flakeflagger = flakify_results_flakeflagger.drop(['Project Name'], axis=1)
flakify_results = pd.concat([flakify_results_IDoFT, flakify_results_flakeflagger])
flakify_results.drop_duplicates(subset=['project_name'], inplace=True)
print(flakify_results.shape)

(310, 5)


In [14]:
# Projects that flakify has already run
results_for_test_projects = pd.concat([test_projects.set_index('project_name'), flakify_results.set_index('project_name')], axis=1, join='inner').reset_index()
print(results_for_test_projects.shape)
results_for_test_projects

(17, 7)


,project_name,Project Address,SHA,Accuracy,F1,Precision,Recall
0,achilles,git@github.com:doanduyhai/Achilles.git,e3099bdce342910951c4862c78751fd81ed4552e,100.000000,100.000000,100.000000,100.000000
1,ambari,git@github.com:apache/ambari.git,NaN,94.000000,84.000000,75.000000,95.000000
2,assertj-core,git@github.com:joel-costigliola/assertj-core.git,NaN,99.000000,40.000000,25.000000,100.000000
3,commons-exec,git@github.com:apache/commons-exec.git,NaN,90.000000,40.000000,25.000000,100.000000
4,dropwizard,git@github.com:dropwizard/dropwizard.git,07dfaed697427e208d65049f80a5d1949833b7cd,100.000000,100.000000,100.000000,100.000000
5,hadoop,git@github.com:apache/hadoop.git,cc2babc1f75c93bf89a8f10da525f944c15d02ea,97.315436,98.630137,98.630137,98.630137
6,hbase,git@github.com:apache/hbase.git,801fc05e9c082ab12c7c1207087d3667ab162d1a,82.692308,90.322581,97.674419,84.000000
7,httpcore,git@github.com:apache/httpcore.git,NaN,98.000000,81.000000,74.000000,90.000000
8,jackrabbit-oak,git@github.com:apache/jackrabbit-oak.git,11985b3fbbd4f7f39dfaf368b01b5e0c67a32f0d,96.000000,97.959184,100.000000,96.000000
9,jimfs,git@github.com:google/jimfs.git,ced6093fe69a31e37ba8bbf63858b50c7164f4a4,100.000000,100.000000,100.000000,100.000000


In [15]:
# Projects that flakify needs to run
additional_projects = test_projects.loc[~test_projects['project_name'].isin(results_for_test_projects['project_name'])]
print(additional_projects)

                     Project Address  SHA project_name
15  git@github.com:togglz/togglz.git  NaN       togglz


# Flakify Test case extract

In [2]:
import javalang
import glob
import os
import pathlib

# basedir = '../test-projects/projects/'
# count = 0
# def dfs(dir):
#   global count
#   if os.path.isdir(dir):
#     for f in os.listdir(dir):
#       dfs(dir+'/'+f)
#   elif pathlib.Path(dir).suffix == '.java':
#     count += 1
#     print(dir)
#     try:
#       file = open(dir).read()
#       tree = javalang.parse.parse(file)
#       for path, node in tree.filter(javalang.tree.Annotation):
#         print(path, node)
#     except:
#       print('skip', dir)    

# for index, project in additional_projects.iterrows():
#   print('Processing:',project['project_name'])
#   project_name = project['project_name']
#   for f in os.listdir(basedir+project_name):
#     dfs(basedir + project_name +'/'+ f)
#   break
# print(count)

In [40]:


for i, r in project_flaky_tests.iterrows():
  test_class = r['Test_class']
  test_method = r['Test_method']
  print(pathlib.Path(test_class).suffix)


.ZookeeperStateRepositoryTest
.SystemPropertyActivationStrategyTest
.SystemPropertyActivationStrategyTest
.ManagedFeatureManagerTest
.CDIBasicOperationTest
.GuiceIntegrationTest
.JSFMapTest
.ServletBasicOperationTest
.CacheRepositoryTest
.JDBCRepositoryTest
.ThreadBasedUsersTest
.HttpServletRequestHolderTest
.ShiroUsersTest
.SpringBasicOperationTest
.SpringEarlyFeatureUsageTest
.ManagedFeatureManagerTest
.FeatureProxyTest
.SystemPropertyActivationStrategyTest
.CassandraStateRepositoryTest
.CassandraStateRepositoryTest
.CassandraStateRepositoryTest
.CassandraStateRepositoryTest
.CassandraStateRepositoryTest
.SystemPropertyActivationStrategyTest
.SystemPropertyActivationStrategyTest
.SystemPropertyActivationStrategyTest
.SystemPropertyActivationStrategyTest
.ZookeeperStateRepositoryTest
